## Zip Code

In [18]:
import mywebdriver as webdriver
import mywebdriver
import pandas as pd


### Price by zipcode

In [19]:
url = 'https://www.zip-codes.com/state/tx.asp'
soup = webdriver.get_soup(url, driver_type='selenium')

In [20]:
links = soup.select('table.statTable>tbody>tr>td:nth-of-type(2)>a')

In [21]:
links = soup.select('table.statTable>tbody>tr>td:first-of-type>a')
zipcode_tx = [link.text.split()[-1] for link in links]
links = soup.select('table.statTable>tbody>tr>td:nth-of-type(2)>a')
city_tx = [link.text.split()[-1] for link in links]
links = soup.select('table.statTable>tbody>tr>td:nth-of-type(3)>a')
county_tx = [link.text.split()[-1] for link in links]
links = soup.select('table.statTable>tbody>tr>td:nth-of-type(4)')
ziptype = [link.text.split()[-1] for link in links]
ziptype =  ziptype[1:]
dic_data = {'zipcode': zipcode_tx, 'city': city_tx, 'county': county_tx, 'ziptype': ziptype}

In [22]:
df_zip = pd.DataFrame(dic_data, columns=['zipcode', 'city', 'county', 'ziptype'])
df_zip.to_csv('./data/zipcode_tx.csv', index=False)
df_zip.ziptype.unique()
df_zip = df_zip[df_zip.ziptype=='Standard']

### Redfin data by zip code

In [23]:
import sys
sys.path.append('./')
hou_counties = ['Harris', 'Bend', 'Montgomery', 'Walker']
df_zip_hou = df_zip[df_zip.county.isin(hou_counties)]
df_zip_hou.shape


(175, 4)

In [24]:
print (df_zip_hou)

     zipcode       city  county   ziptype
1026   77002    Houston  Harris  Standard
1027   77003    Houston  Harris  Standard
1028   77004    Houston  Harris  Standard
1029   77005    Houston  Harris  Standard
1030   77006    Houston  Harris  Standard
...      ...        ...     ...       ...
1403   77562  Highlands  Harris  Standard
1409   77571      Porte  Harris  Standard
1422   77586   Seabrook  Harris  Standard
1423   77587    Houston  Harris  Standard
1429   77598    Webster  Harris  Standard

[175 rows x 4 columns]


In [25]:

import mywebdriver
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from random import random
import traceback
import time

In [11]:
# TEST
zipcode = '77007'
url_redfin = f'https://www.redfin.com/zipcode/{zipcode}/housing-market'

In [12]:
chrome_options = Options()
chrome_options.add_argument("--window-size=2400,1080")
chrome_options.add_argument("--silent") # open without popup
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get(url_redfin)

/var/folders/ds/f3qghlvs6z51c03c15bdzhs80000gn/T/ipykernel_1341/340231036.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [13]:
driver.find_element_by_css_selector('div[data-react-server-root-id="19"] .ModeToggler.graphTabs button:first-of-type').click()
svg = driver.find_element_by_css_selector('div[data-react-server-root-id="19"] .lineGraph .VictoryContainer>svg')
svg = svg.get_attribute('outerHTML')
svg = svg.replace('svg width', 'svg xmlns="http://www.w3.org/2000/svg" width')
with open(f'image/{zipcode}_1y.svg', 'w') as f:
    f.write(svg)
driver.find_element_by_css_selector('div[data-react-server-root-id="19"] .ModeToggler.graphTabs button:nth-of-type(3)').click()
svg3 = driver.find_element_by_css_selector('div[data-react-server-root-id="19"] .lineGraph .VictoryContainer>svg')
svg3 = svg3.get_attribute('outerHTML')
svg3 = svg3.replace('svg width', 'svg xmlns="http://www.w3.org/2000/svg" width')
with open(f'image/{zipcode}_3y.svg', 'w') as f:
    f.write(svg3)

In [27]:
ls_zipdata = []
ls_err = []
sec_sleep = 20
for i, row in df_zip_hou.iterrows():
    # initial zip code 
    zipcode = row['zipcode']
    url_redfin = f'https://www.redfin.com/zipcode/{zipcode}/housing-market'
    try:
        # load web
        chrome_options = Options()
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--silent") # open without popup
        driver = webdriver.Chrome()
        driver.get(url_redfin)
        # get price trend svg
        driver.find_element_by_css_selector('div[data-react-server-root-id="19"] .ModeToggler.graphTabs button:first-of-type').click()
        svg = driver.find_element_by_css_selector('div[data-react-server-root-id="19"] .lineGraph .VictoryContainer>svg')
        svg = svg.get_attribute('outerHTML')
        svg = svg.replace('svg width', 'svg xmlns="http://www.w3.org/2000/svg" width')
        with open(f'image/{zipcode}_1y.svg', 'w') as f:
            f.write(svg)
        driver.find_element_by_css_selector('div[data-react-server-root-id="19"] .ModeToggler.graphTabs button:nth-of-type(3)').click()
        svg3 = driver.find_element_by_css_selector('div[data-react-server-root-id="19"] .lineGraph .VictoryContainer>svg')
        svg3 = svg3.get_attribute('outerHTML')
        svg3 = svg3.replace('svg width', 'svg xmlns="http://www.w3.org/2000/svg" width')
        with open(f'image/{zipcode}_3y.svg', 'w') as f:
            f.write(svg3)

        # load market insight # soup_zip = mywebdriver.get_soup(url_redfin, driver_type='selenium')
        soup_zip = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()
        # median price, # of home sold, median DOM, score, 
        soup_price = soup_zip.select('.MarketInsightsGraphSection:first-of-type button:first-of-type')[0]
        soup_sold = soup_zip.select('.MarketInsightsGraphSection:first-of-type button:nth-of-type(2)')[0]
        soup_DOM = soup_zip.select('.MarketInsightsGraphSection:first-of-type button:nth-of-type(3)')[0]
        median_price = float(soup_price.select('.dataPoints>.value')[0].text.replace('$','').replace(',',''))
        median_price_yoy = float(soup_price.select('.dataPoints>.yoyChange')[0].text.replace('+','').replace('%','').replace(' year-over-year', ''))
        n_sold = float(soup_sold.select('.dataPoints>.value')[0].text.replace('$','').replace(',',''))
        n_sold_yoy = float(soup_sold.select('.dataPoints>.yoyChange')[0].text.replace('+','').replace('%','').replace(' year-over-year', ''))
        DOM = float(soup_DOM.select('.dataPoints>.value')[0].text.replace('$','').replace(',',''))
        DOM_yoy = float(soup_DOM.select('.dataPoints>.yoyChange')[0].text.replace('+','').replace('%','').replace(' year-over-year', ''))
        score = int(soup_zip.select('.scoreTM>.score')[0].text)
        dic_zipdata = {'zipcode': zipcode, 'url': url_redfin, 'median_price': median_price, 'median_price_yoy': median_price_yoy, 'n_sold': n_sold, 'n_sold_yoy': n_sold_yoy, 'DOM': DOM, 'DOM_yoy': DOM_yoy, 'redfin_score': score}
        ls_zipdata.append(dic_zipdata)
    except:
        ls_err.append({'zipcode': zipcode, 'url': url_redfin, 'err': traceback.format_exc()})
        if len(ls_err) % 5 == 0:
            print(f'---- There are {len(ls_err)} errors ----')
    time.sleep(random() * sec_sleep)

KeyboardInterrupt: 

In [15]:
# analyze error
df_err = pd.DataFrame(ls_err)
df_err = df_err[['zipcode', 'url', 'err']]
df_err.to_csv('./data/zipcode_data_err.csv', index=False)
df_err.head()

KeyError: "None of [Index(['zipcode', 'url', 'err'], dtype='object')] are in the [columns]"

In [185]:
# check error
len(ls_err)

21

In [190]:
# form dataframe
df_zipdata = pd.DataFrame(ls_zipdata)
df_zipdata = df_zipdata[['zipcode', 'url', 'median_price', 'median_price_yoy', 'n_sold', 'n_sold_yoy', 'DOM', 'DOM_yoy', 'redfin_score']]
df_zipdata['median_price_yoy'] = df_zipdata['median_price_yoy']/100
df_zipdata['n_sold_yoy'] = df_zipdata['n_sold_yoy']/100
df_zipdata['DOM_yoy'] = df_zipdata['DOM_yoy']/100
df_zipdata.shape

(154, 9)

In [194]:
# get image path
def get_image_path(x):
    return f'./image/{x}_1y.svg'
def get_image_path3(x):
    return f'./image/{x}_3y.svg'
df_zipdata['svg1'] = df_zipdata.zipcode.map(get_image_path)
df_zipdata['svg3'] = df_zipdata.zipcode.map(get_image_path3)

In [196]:
# save csv
df_zipdata.to_csv('./data/zipcode_data.csv', index=False)